In [14]:
# Importing dependencies

import os
import re
import pandas as pd
from tqdm import tqdm

In [2]:
# Constants
DATA_PATH = 'data/trec07p/data'
LABEL_PATH = 'data/trec07p/full/index'

In [3]:
# Reading the labels
labels = pd.read_csv(LABEL_PATH, sep=' ', header=None)

labels.head()

,0,1
0,spam,../data/inmail.1
1,ham,../data/inmail.2
2,spam,../data/inmail.3
3,spam,../data/inmail.4
4,spam,../data/inmail.5


In [4]:
# Dropping any na values
labels.dropna(inplace=True)

# Adding columns to the DF
labels.columns = ['label', 'id']

# Chaging to word spam and ham to 1 and 0
labels['label'] = labels['label'].apply(lambda x: 1 if x=='spam' else 0)

# For id column, storing just the id
labels['id'] = labels['id'].apply(lambda x: x.split('/')[2])

labels.head()

,label,id
0,1,inmail.1
1,0,inmail.2
2,1,inmail.3
3,1,inmail.4
4,1,inmail.5


In [5]:
# Saving the file to a CSV file
labels.to_csv('data/processed_labels.csv', index=False)
del labels

In [6]:
# Making a list of all possible emails
emails = os.listdir(DATA_PATH)

emails[:10]

['inmail.57004',
 'inmail.7847',
 'inmail.64696',
 'inmail.60936',
 'inmail.53892',
 'inmail.73981',
 'inmail.36762',
 'inmail.32477',
 'inmail.54242',
 'inmail.41862']

In [7]:
len(emails)

75419

In [15]:
subjects = []
for file in tqdm(emails):
    with open(os.path.join(DATA_PATH, file), "r", encoding='ISO-8859-1') as f:
        text = f.read()
        
        lines = text.split('\n')
        
        # Extracting the subject
        message = '';
        for line in lines:
            if line.find('Subject: ') > -1:
                subjects.append(line[9:])
            
            if not re.findall(r".:.", line):
                message += line + "\n"
            
        break

  0%|          | 0/75419 [00:00<?, ?it/s]


In [17]:
message

'\tby flax9.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l5DNj8L9013945\n\ttests=BAYES_50,DKIM_SIGNED,DKIM_VERIFIED,DK_POLICY_TESTING,DK_SIGNED,DK_VERIFIED,SPF_PASS\n  s=s1024; d=yahoo.com;\n  b=OjsU7eqARMmDhheCvrx8Da6Xv/aVpFb8M32LsEXjz7GYZ6JfyEpD4gim5psa67k9rUy0V/WaKT45stoHZ6VNXKMFCCUAGx+xZat6KsucFahBtSNJ6qSla0FSOJi+ZHL8v7KxYK57MTYb2NeHFSaC2F/tSoTtBiq+mQvbmcSdp10=;\n\n> \n> #!/usr/bin/perl -w\n> \n> use strict;\n> \n> my $program = "vi";\n> my $status  = `/bin/ps cat | /bin/grep $program`;\n> \n> if ( length($status) > 0 ) {\n>     print "$status";                       #extract\n> pid from here\n> }\n> else { print "$program not running\\n" }    # start\n> program\n> \n> ================================================\n>\n\nThis will work, in the past I have used \n\nand \n\n\n\n       \n____________________________________________________________________________________\nGet the free Yahoo! toolbar and rest assured with the added security of spyware protection.\n\n-- \n\n\n\n'